In [ ]:
import pandas as pd
from collections import defaultdict
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix

In [ ]:
dfx = pd.read_csv("dfx.csv")

Deleting one row MBC

In [ ]:
dfx_noMBC = dfx[dfx['Oncotree Code']!='MBC']

In [ ]:
dfx.head()

In [ ]:
dfx_only_onc = dfx_noMBC.drop(['Unnamed: 0','Cancer Type Detailed','ER Status','HER2 Status','Neoplasm Histologic Grade','Tumor Stage','PR Status','Tumor Size'],axis=1)

In [ ]:
dfx_only_onc.head()
#Where OncoTreeCode is NA
# dfx_only_onc = dfx_only_onc[np.isfinite(dfx_only_onc['Oncotree Code'])]
# dfx_only_onc['Oncotree Code'].isnull().values.any()
dfx_only_onc = dfx_only_onc[pd.notnull(dfx_only_onc['Oncotree Code'])]
dfx_only_onc['Oncotree Code'].isnull().values.any()

In [ ]:
dfx_only_onc.head()

In [ ]:
X = dfx_only_onc.loc[:, dfx_only_onc.columns != 'Oncotree Code']
Y = dfx_only_onc['Oncotree Code']
X

In [ ]:
Y

Finding number of unique values

In [ ]:
np.unique(Y, return_counts=True)

Imputing missing values

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X_impu = imp.fit_transform(X)
X_impu = pd.DataFrame(X_impu, columns=X.columns)


In [ ]:
X_impu

Feature Selection

# Using 2000 features

In [ ]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
selector = SelectKBest(chi2, k=2000)
selector.fit(X_impu, Y)
cols = selector.get_support(indices=True)
X_new = X_impu.iloc[:,cols]


In [ ]:
data2k = X_new
data2k['label'] = Y
corr = data2k.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [ ]:
print("the shape of new data is: ",X_new.shape)

Resampling

In [ ]:
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state=42,k_neighbors=3, sampling_strategy='auto')
X_s, y_s = sm.fit_resample(X_new, Y)
print('Resampled dataset shape after running SMOTE algorithm for Data Oversampling %s' % Counter(y_s))
# X_s['y'] = y_s
dfffff = pd.DataFrame(X_s)
dfffff = pd.concat([dfffff,y_s],axis=1)
sns.countplot(dfffff['Oncotree Code'])

In [ ]:
sns.countplot(dfx['Oncotree Code'])

Training Testing Split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(X_s, y_s, test_size=0.2)
print ("the shape of training data is ",X_train.shape, y_train.shape)
print ("the shape of testing data is ",X_test.shape, y_test.shape)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc

In [ ]:
clf = SVC(kernel="linear", C=0.025)
#79 before
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(multilabel_confusion_matrix(y_test, y_pred, labels=['IDC','ILC','MDLC','BREAST','IMMC']))
score = clf.score(X_test, y_test)

print(score)


In [ ]:
clf = SVC(kernel='rbf', gamma=0.001)
 #81.21 before, 0.715 0.05 gamma, 99.006 0.001 gamma
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(multilabel_confusion_matrix(y_test, y_pred, labels=['IDC','ILC','MDLC','BREAST','IMMC']))
score = clf.score(X_test, y_test)
print(score)

In [ ]:
clf = LogisticRegression()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(multilabel_confusion_matrix(y_test, y_pred, labels=['IDC','ILC','MDLC','BREAST','IMMC']))
score = clf.score(X_test, y_test)
print(score)

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(multilabel_confusion_matrix(y_test, y_pred, labels=['IDC','ILC','MDLC','BREAST','IMMC']))
score = clf.score(X_test, y_test)
print(score)

In [ ]:
clf = GaussianNB()
#0.568 before
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(multilabel_confusion_matrix(y_test, y_pred, labels=['IDC','ILC','MDLC','BREAST','IMMC']))
score = clf.score(X_test, y_test)
print(score)

### Ensemble 

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
lr = LogisticRegression(max_iter = 20)
svm_linear = SVC(kernel='linear',C=0.025)
rf = RandomForestClassifier()
svm = SVC(kernel = 'rbf', gamma=0.001)
evc = VotingClassifier( estimators= [('svm_linear',svm_linear),('lr',lr),('svm',svm),('rf',rf)], voting = 'hard')
evc.fit(X_train,y_train)
evc.score(X_test, y_test)

In [ ]:
y_pred = evc.predict(X_test)
print(multilabel_confusion_matrix(y_test, y_pred, labels=['IDC','ILC','MDLC','BREAST','IMMC']))


# Using 5000 features

In [ ]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
selector = SelectKBest(chi2, k=5000)
selector.fit(X_impu, Y)
cols = selector.get_support(indices=True)
X_new = X_impu.iloc[:,cols]

In [ ]:
print("the shape of new data is: ",X_new.shape)

Resampling

In [ ]:
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state=42,k_neighbors=3, sampling_strategy='auto')
X_s, y_s = sm.fit_resample(X_new, Y)
print('Resampled dataset shape after running SMOTE algorithm for Data Oversampling %s' % Counter(y_s))

Training Testing Split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(X_s, y_s, test_size=0.2)
print ("the shape of training data is ",X_train.shape, y_train.shape)
print ("the shape of testing data is ",X_test.shape, y_test.shape)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(multilabel_confusion_matrix(y_test, y_pred, labels=['IDC','ILC','MDLC','BREAST','IMMC']))
score = clf.score(X_test, y_test)
print(score)

In [ ]:
feature_importances = zip(cols, clf.feature_importances_)
feature_importances = sorted(feature_importances, key = lambda x:x[1],reverse = True)
for x, y in feature_importances[:10]:
    print (dfx.columns[x],y)

In [ ]:
clf = SVC(kernel="linear", C=0.025)
#79 before
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(multilabel_confusion_matrix(y_test, y_pred, labels=['IDC','ILC','MDLC','BREAST','IMMC']))
score = clf.score(X_test, y_test)
print(score)

In [ ]:
clf = SVC(kernel='rbf', gamma=0.001)
 #81.21 before, 0.715 0.05 gamma, 99.006 0.001 gamma
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(multilabel_confusion_matrix(y_test, y_pred, labels=['IDC','ILC','MDLC','BREAST','IMMC']))
score = clf.score(X_test, y_test)
print(score)

In [ ]:
clf = LogisticRegression()

clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)
y_pred = clf.predict(X_test)
print(multilabel_confusion_matrix(y_test, y_pred, labels=['IDC','ILC','MDLC','BREAST','IMMC']))
print(score)

In [ ]:
clf = GaussianNB()
#0.568 before
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(multilabel_confusion_matrix(y_test, y_pred, labels=['IDC','ILC','MDLC','BREAST','IMMC']))
score = clf.score(X_test, y_test)

print(score)

### Ensemble 

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier


In [ ]:
lr = LogisticRegression(max_iter = 20)
rf = RandomForestClassifier()

svm_linear = SVC(kernel='linear',C=0.025)
svm = SVC(kernel = 'rbf', gamma=0.001)
evc = VotingClassifier( estimators= [('svm_linear',svm_linear),('lr',lr),('svm',svm),('rf',rf)], voting = 'hard')
evc.fit(X_train,y_train)
evc.score(X_test, y_test)

In [ ]:
y_pred = evc.predict(X_test)
print(multilabel_confusion_matrix(y_test, y_pred, labels=['IDC','ILC','MDLC','BREAST','IMMC']))


# Dimensionality Reduction and Plot

In [ ]:
from sklearn.decomposition import KernelPCA,PCA
pca = PCA(n_components=3)
X_reduced = pca.fit_transform(X_impu)
X_reduced.shape

In [ ]:
y_num = Y.apply(['IDC','ILC','MDLC','BREAST','IMMC'].index)

In [ ]:
x_nen=pd.DataFrame(data=X_reduced[0:,0:],index=[i for i in range(X_reduced.shape[0])],columns=['f'+str(i) for i in range(X_reduced.shape[1])])

In [ ]:
x_nen['y'] = y_num

In [ ]:
idcs = x_nen[x_nen.y == 0].iloc[:500,:]
ilcs = x_nen[x_nen.y == 1]
mdlcs = x_nen[x_nen.y == 2]
brs = x_nen[x_nen.y == 3]
immcs = x_nen[x_nen.y == 4]
# mdlcs.shape

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')
ax.scatter(idcs.iloc[:,0],idcs.iloc[:,1],idcs.iloc[:,2],c='r',marker='^',label='IDC')
ax.scatter(ilcs.iloc[:,0],ilcs.iloc[:,1],ilcs.iloc[:,2],c='green',marker='v',label='ILC')
ax.scatter(mdlcs.iloc[:,0],mdlcs.iloc[:,1],mdlcs.iloc[:,2],c='blue',marker='o',label='MDLC')
ax.scatter(brs.iloc[:,0],brs.iloc[:,1],brs.iloc[:,2],c='black',marker='d',label='BREAST')
ax.scatter(immcs.iloc[:,0],immcs.iloc[:,1],immcs.iloc[:,2],c='yellow',marker='>',label='IMMC')


ax.view_init(40, 90)
ax.legend()
plt.savefig('OncoPCA.png')

# Using resampled data

In [ ]:
from sklearn.decomposition import KernelPCA,PCA
pca = PCA(n_components=3)
X_reduced = pca.fit_transform(X_s)
X_reduced.shape

In [ ]:
y_num = y_s.apply(['IDC','ILC','MDLC','BREAST','IMMC'].index)

In [ ]:
x_nen=pd.DataFrame(data=X_reduced[0:,0:],index=[i for i in range(X_reduced.shape[0])],columns=['f'+str(i) for i in range(X_reduced.shape[1])])

In [ ]:
y_num

In [ ]:
x_nen['y'] = y_num

In [ ]:
idcs = x_nen[x_nen.y == 0]
ilcs = x_nen[x_nen.y == 1]
mdlcs = x_nen[x_nen.y == 2]
brs = x_nen[x_nen.y == 3]
immcs = x_nen[x_nen.y == 4]
ilcs.shape

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')
ax.scatter(idcs.iloc[:,0],idcs.iloc[:,1],idcs.iloc[:,2],c='r',marker='^',label='IDC')
ax.scatter(ilcs.iloc[:,0],ilcs.iloc[:,1],ilcs.iloc[:,2],c='green',marker='v',label='ILC')
ax.scatter(mdlcs.iloc[:,0],mdlcs.iloc[:,1],mdlcs.iloc[:,2],c='blue',marker='o',label='MDLC')
ax.scatter(brs.iloc[:,0],brs.iloc[:,1],brs.iloc[:,2],c='black',marker='d',label='BREAST')
ax.scatter(immcs.iloc[:,0],immcs.iloc[:,1],immcs.iloc[:,2],c='yellow',marker='>',label='IMMC')


ax.view_init(50, 20)

ax.legend(scatterpoints=1,
           loc='lower left',
           ncol=3,
           fontsize=8)

plt.savefig('OncoSMOTE.png')